# 🎬 Hệ gợi ý (Model-based CF) với ALS (Explicit)

Notebook này huấn luyện **ALS tường minh (Explicit ALS)** cho bài toán dự đoán điểm phim và xuất dự đoán theo định dạng **`Id,Score`** (bắt đầu `Id` từ 1) giống file mẫu.

**Quy trình**
1. Đọc `train.txt` / `test.txt`
2. Ánh xạ ID gốc → chỉ số liên tục (0..n-1)
3. Tính **baseline** (μ, độ lệch người dùng bᵤ, độ lệch item bᵢ) để dự phòng cold-start
4. Xây ma trận thưa user–item
5. Huấn luyện **ALS tường minh** (tối thiểu hóa sai số bình phương + L2)
6. (Tuỳ chọn) Đánh giá nhanh RMSE trên tập holdout
7. Dự đoán cho test và xuất `submission.csv`


In [ ]:

import os
import numpy as np
import pandas as pd
from scipy import sparse
from pathlib import Path
from typing import Optional, Tuple

# --- Đường dẫn (sửa nếu cần) ---
TRAIN_PATH = "data/train.txt"   # ví dụ: "/kaggle/input/.../train.txt"
TEST_PATH  = "data/test.txt"    # ví dụ: "/kaggle/input/.../test.txt"
SUB_PATH   = "output/CF_ALS_submission.csv"   # nơi lưu kết quả dự đoán

print(Path(TRAIN_PATH).resolve())
print(Path(TEST_PATH).resolve())


In [ ]:

# --- Đọc dữ liệu ---
df_train = pd.read_csv(
    TRAIN_PATH,
    sep=r"\s+",
    header=None,
    names=["userid","movieid","rating"],
    engine="python",
)
df_test = pd.read_csv(
    TEST_PATH,
    sep=r"\s+",
    header=None,
    names=["userid","movieid"],
    engine="python",
)

print(df_train.shape, df_test.shape)
display(df_train.head())
display(df_test.head())


In [ ]:

# --- Ánh xạ ID gốc → chỉ số liên tục (0..n-1), gộp cả train & test để tránh thiếu ---
all_u = pd.concat([df_train["userid"], df_test["userid"]], ignore_index=True)
all_i = pd.concat([df_train["movieid"], df_test["movieid"]], ignore_index=True)

uid_uniques = all_u.drop_duplicates()
iid_uniques = all_i.drop_duplicates()
uid_map = pd.Series(np.arange(len(uid_uniques), dtype=np.int32), index=uid_uniques.values)
iid_map = pd.Series(np.arange(len(iid_uniques), dtype=np.int32), index=iid_uniques.values)

df_train["u_idx"] = df_train["userid"].map(uid_map).astype(np.int32)
df_train["i_idx"] = df_train["movieid"].map(iid_map).astype(np.int32)
df_test["u_idx"]  = df_test["userid"].map(uid_map).astype(np.int32)
df_test["i_idx"]  = df_test["movieid"].map(iid_map).astype(np.int32)

n_users = uid_map.size
n_items = iid_map.size
print(f"n_users={n_users}, n_items={n_items}, train_rows={len(df_train)}, test_rows={len(df_test)}")


In [ ]:

# --- Baseline (μ + b_u + b_i) dùng làm dự phòng (fallback) cho cold-start ---
mu = df_train["rating"].mean()
lambda_reg_bias = 10.0

# Ma trận tạm để tính nhanh chỉ mục người dùng/item đã có rating
R_tmp = sparse.coo_matrix(
    (df_train["rating"].astype(np.float32), (df_train["u_idx"], df_train["i_idx"])),
    shape=(n_users, n_items),
).tocsr()

users_items = [R_tmp[u].indices for u in range(n_users)]
items_users = [R_tmp[:, i].indices for i in range(n_items)]

b_u = np.zeros(n_users, dtype=np.float32)
b_i = np.zeros(n_items, dtype=np.float32)

# Lặp vài vòng coordinate-descent đơn giản
for _ in range(3):
    # cập nhật b_u
    for u in range(n_users):
        idx_i = users_items[u]
        if len(idx_i) == 0: 
            b_u[u] = 0.0
            continue
        r_ui = R_tmp[u, idx_i].toarray().ravel()
        b_u[u] = ((r_ui - mu - b_i[idx_i]).sum()) / (lambda_reg_bias + len(idx_i))
    # cập nhật b_i
    for i in range(n_items):
        idx_u = items_users[i]
        if len(idx_u) == 0:
            b_i[i] = 0.0
            continue
        r_ui = R_tmp[idx_u, i].toarray().ravel()
        b_i[i] = ((r_ui - mu - b_u[idx_u]).sum()) / (lambda_reg_bias + len(idx_u))

def predict_baseline(u: int, i: int, clip: Optional[Tuple[float,float]]=(1,5)) -> float:
    """Dự đoán theo baseline: μ + b_u + b_i (có clip về [1,5] nếu cần)."""
    val = mu + b_u[u] + b_i[i]
    if clip is not None:
        val = float(np.clip(val, clip[0], clip[1]))
    return float(val)

print("Baseline sẵn sàng. Global mean:", mu)


In [ ]:

# --- Tách nhanh một tập holdout để ước lượng RMSE (tuỳ chọn) ---
rng = np.random.default_rng(2024)
mask = rng.random(len(df_train)) < 0.1  # lấy 10% làm holdout
df_valid = df_train[mask].reset_index(drop=True)
df_tr    = df_train[~mask].reset_index(drop=True)
print("Số dòng train dùng ALS:", df_tr.shape, "| holdout:", df_valid.shape)

# --- Xây ma trận thưa CSR từ df_tr ---
R = sparse.coo_matrix(
    (df_tr["rating"].astype(np.float32), (df_tr["u_idx"], df_tr["i_idx"])),
    shape=(n_users, n_items),
).tocsr()

# Tuỳ chọn: huấn luyện trên dữ liệu đã trừ đi trung bình toàn cục
CENTER = True
r_mean = mu


In [ ]:

# --- Cài đặt ALS tường minh (Explicit ALS) ---
def als_explicit(R_csr: sparse.csr_matrix, 
                 n_users: int, n_items: int, 
                 k: int = 64, 
                 reg: float = 0.1, 
                 n_iters: int = 10,
                 center: bool = True,
                 mean_val: float = 0.0,
                 random_state: int = 42):
    """
    Bài toán: minimize \sum_{(u,i)∈Obs} (r_ui - u_u^T v_i)^2 + reg*(||U||^2 + ||V||^2)
    Nếu center=True, mô hình học trên (r_ui - mean_val) và cộng lại mean_val khi suy luận.
    R_csr: ma trận CSR (n_users x n_items) chỉ chứa các rating đã quan sát.
    """
    rng = np.random.default_rng(random_state)
    U = 0.1 * rng.standard_normal((n_users, k)).astype(np.float32)
    V = 0.1 * rng.standard_normal((n_items, k)).astype(np.float32)

    # Danh sách chỉ mục để truy cập nhanh
    users_items = [R_csr[u].indices for u in range(n_users)]
    items_users = [R_csr[:, i].indices for i in range(n_items)]

    for it in range(n_iters):
        # --- Cập nhật U (cố định V) ---
        VtV = V.T @ V + reg * np.eye(k, dtype=np.float32)
        for u in range(n_users):
            idx_i = users_items[u]
            if len(idx_i) == 0:
                continue
            V_i = V[idx_i]                              # (m, k)
            r_u = R_csr[u, idx_i].toarray().ravel()     # (m,)
            if center:
                r_u = r_u - mean_val
            # Giải (V_i^T V_i + reg I) U_u = V_i^T r_u
            A = VtV - (reg * np.eye(k, dtype=np.float32)) + V_i.T @ V_i + reg * np.eye(k, dtype=np.float32)
            b = V_i.T @ r_u
            U[u] = np.linalg.solve(A, b)

        # --- Cập nhật V (cố định U) ---
        UtU = U.T @ U + reg * np.eye(k, dtype=np.float32)
        for i in range(n_items):
            idx_u = items_users[i]
            if len(idx_u) == 0:
                continue
            U_u = U[idx_u]                              # (m, k)
            r_i = R_csr[idx_u, i].toarray().ravel()     # (m,)
            if center:
                r_i = r_i - mean_val
            # Giải (U_u^T U_u + reg I) V_i = U_u^T r_i
            A = UtU - (reg * np.eye(k, dtype=np.float32)) + U_u.T @ U_u + reg * np.eye(k, dtype=np.float32)
            b = U_u.T @ r_i
            V[i] = np.linalg.solve(A, b)
        print(f"Hoàn tất vòng lặp ALS {it+1}/{n_iters}.")
    return U, V


In [ ]:

# --- Huấn luyện ALS ---
K = 64         # số chiều ẩn
LAMBDA = 0.1   # hệ số L2
N_ITERS = 10   # số vòng lặp ALS

U, V = als_explicit(R, n_users, n_items, k=K, reg=LAMBDA, n_iters=N_ITERS, center=CENTER, mean_val=r_mean, random_state=2025)
print("Kích thước nhân tố:", U.shape, V.shape)


In [ ]:

# --- Ước lượng RMSE trên tập holdout (tuỳ chọn) ---
def predict_dot(u, i, U, V, center=True, mean_val=0.0):
    val = float(U[u] @ V[i])
    if center:
        val += mean_val
    return val

def rmse_holdout(df_valid, U, V, clip=(1,5), center=True, mean_val=0.0):
    se = 0.0
    n = 0
    for u, i, r in df_valid[["u_idx","i_idx","rating"]].itertuples(index=False):
        pred = predict_dot(u, i, U, V, center=center, mean_val=mean_val)
        if clip is not None:
            pred = np.clip(pred, clip[0], clip[1])
        se += (pred - r) ** 2
        n += 1
    return np.sqrt(se / max(n, 1))

val_rmse = rmse_holdout(df_valid, U, V, clip=(1,5), center=CENTER, mean_val=r_mean)
print(f"Holdout RMSE: {val_rmse:.5f}")


In [ ]:

# --- Dự đoán cho test kèm fallback về baseline nếu gặp cold-start ---
def predict_with_fallback(u, i, U, V, clip=(1,5)) -> float:
    has_u = np.any(U[u]) if (0 <= u < U.shape[0]) else False
    has_i = np.any(V[i]) if (0 <= i < V.shape[0]) else False
    if has_u and has_i:
        val = float(U[u] @ V[i])
        if CENTER:
            val += r_mean
        if clip is not None:
            val = float(np.clip(val, clip[0], clip[1]))
        return val
    # Nếu thiếu user/item trong nhân tố, quay về baseline
    return predict_baseline(u, i, clip=clip)

CLIP = (1, 5)
preds = np.empty(len(df_test), dtype=np.float32)
for idx, (u, i) in enumerate(zip(df_test["u_idx"].values, df_test["i_idx"].values)):
    preds[idx] = predict_with_fallback(u, i, U, V, clip=CLIP)

print("Ví dụ 10 dự đoán đầu:", preds[:10])


In [ ]:

# --- Xuất file submission theo định dạng mẫu (Id bắt đầu từ 1) ---
submission = pd.DataFrame({
    "Id": np.arange(1, len(df_test) + 1, dtype=np.int64),
    "Score": preds
})
submission.to_csv(SUB_PATH, index=False, float_format="%.6f")
print("Đã lưu:", SUB_PATH, submission.shape)
display(submission.head())
